In [45]:
import numpy as np
import pandas as pd
import os
import sys
from scipy.stats import ttest_rel
from sklearn.metrics import r2_score

In [2]:
INFERRED_PREDICATE_FILE_NAME = "SERIES.txt"
TRUTH_PREDICATE_FILE_NAME = "Series_truth.txt"
AR_BASELINE_FILE_NAME = "ARBaseline_obs.txt"
CLUSTER_BASELINE_NAME = "ARBaselineNaiveTD_obs.txt"

In [49]:
METRICS = ["NRMSE", "SMAPE", "MASE", "R2"]

In [15]:
def mase(a, f, scale):
    return np.mean(np.abs(a - f)) / scale

def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [46]:
def eval_run(data_dir, result_dir, n_folds, method_name, results_df, params, eval_baseline=False, cluster_baseline=False):
    
    mase_scale_factors = dict()

    for line in open(os.path.join(data_dir, "mase_scale.txt"), "r").readlines():
        tokens = line.rstrip().split("\t")
        mase_scale_factors[tokens[0]] = float(tokens[1])

    stddevs = dict()

    for line in open(os.path.join(data_dir, "stddevs.txt"), "r").readlines():
        tokens = line.rstrip().split("\t")
        stddevs[tokens[0]] = float(tokens[1])
        
    result_rows = []    
    
    for i in range(30):
        fold_dir = str(i).zfill(3)
        if not os.path.isdir(os.path.join(result_dir, fold_dir)):
            continue

        result_fold_dir = os.path.join(result_dir, fold_dir)
        truth_fold_dir = os.path.join(data_dir, fold_dir)

        truth_lines = open(os.path.join(truth_fold_dir, TRUTH_PREDICATE_FILE_NAME), "r").readlines()

        if eval_baseline:
            result_lines = open(os.path.join(truth_fold_dir, AR_BASELINE_FILE_NAME), "r").readlines()
        else:
            result_lines = open(os.path.join(result_fold_dir, INFERRED_PREDICATE_FILE_NAME), "r").readlines()

        if cluster_baseline:
            result_lines = open(os.path.join(truth_fold_dir, CLUSTER_BASELINE_NAME), "r").readlines()


        truth_dict = dict()
        result_dict = dict()
        ar_baseline_dict = dict()

        for line in truth_lines:
            tokens = line.split("\t")
            series_id = tokens[0]
            timestep = tokens[1]
            val = tokens[2].rstrip()

            if series_id not in truth_dict:
                truth_dict[series_id] = dict()

            truth_dict[series_id][timestep] = float(val)

        for line in result_lines:
            tokens = line.split("\t")
            series_id = tokens[0]
            timestep = tokens[1]
            val = tokens[2].rstrip()

            if series_id not in result_dict:
                result_dict[series_id] = dict()

            result_dict[series_id][timestep] = float(val)

            
        for series in truth_dict.keys():
            abs_errors = []

            sq_errors = []

            truth_values = []
            
            predicted_values = []

            for timestep in sorted(list(truth_dict[series].keys())):
                truth_values += [truth_dict[series][timestep]]
                predicted_values += [result_dict[series][timestep]]

                ts_abs_error = np.abs(truth_dict[series][timestep] - result_dict[series][timestep])
                abs_errors += [ts_abs_error]
                sq_errors += [ts_abs_error ** 2]

            s_mase = mase(np.array(truth_values), np.array(predicted_values), mase_scale_factors[series])
            
            s_smape = smape(np.array(truth_values), np.array(predicted_values))

            s_nrmse = np.sqrt(np.mean(sq_errors)) / stddevs[series]

            r2 = r2_score(np.array(truth_values), np.array(predicted_values))

            result_rows += [{"base_noise_std": params[0], "clus_or_std": params[1], "temp_or_std": params[2], "Series_ID": series, "Forecast_Window": fold_dir, "Method": method_name, "NRMSE": s_nrmse,
                                                              "MASE": s_mase, "SMAPE": s_smape, "R2": r2}]
            
    return pd.DataFrame(result_rows)

In [52]:
# E1: no base noise, vary cluster oracle noise
# Hard constraint, weighted constraints of 10

models = ["cw_hard"]
base_noise_stds = [0, 0.5, 1, 1.5, 2]

n_folds = 30

results_df = pd.DataFrame(columns=["base_noise_std", "clus_or_std", "temp_or_std", "Series_ID", "Forecast_Window", "Method", "NRMSE", "MASE"])

for base_noise_std in base_noise_stds:
    temp_or_std = 0
    clus_or_std = 0
    
    for model in models:
        print("Evaluating " + model + " with base noise " + str(base_noise_std) + " and cluster oracle noise std. " + str(clus_or_std))
        
        data_dir = "data/E2_basenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_" + str(temp_or_std) + "/window_size_4/eval"
        result_dir = "results/Online/E2_basenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_" + str(temp_or_std) + "/window_size_4/" + model + "/inferred-predicates"

        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "PSL_" + str(model), results_df, [base_noise_std, clus_or_std, temp_or_std])])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "cluster_AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True, cluster_baseline=True)])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True)])

Evaluating cw_hard with base noise 0 and cluster oracle noise std. 0
Evaluating cw_hard with base noise 0.5 and cluster oracle noise std. 0
Evaluating cw_hard with base noise 1 and cluster oracle noise std. 0
Evaluating cw_hard with base noise 1.5 and cluster oracle noise std. 0


/tmp/ipykernel_476139/953914373.py:5: RuntimeWarning: invalid value encountered in divide
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)
/tmp/ipykernel_476139/953914373.py:5: RuntimeWarning: invalid value encountered in divide
  return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)


Evaluating cw_hard with base noise 2 and cluster oracle noise std. 0


In [54]:
for metric in METRICS:
    for model in models:
        for base_noise_std in base_noise_stds:
            window_vals = []
            cluster_ar_window_vals = []
            pure_ar_window_vals = []

            for name, group in results_df[(results_df["Method"] == "PSL_" + str(model)) & (results_df["base_noise_std"] == base_noise_std) & (results_df["clus_or_std"] == 0)].groupby(by=["Forecast_Window"]):
                window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "cluster_AR") & (results_df["base_noise_std"] == base_noise_std) & (results_df["clus_or_std"] == 0)].groupby(by=["Forecast_Window"]):
                cluster_ar_window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "AR") & (results_df["base_noise_std"] == base_noise_std) & (results_df["clus_or_std"] == 0)].groupby(by=["Forecast_Window"]):
                pure_ar_window_vals += [np.mean(group[metric].values)]

            print("Base_oracle_noise_std = " + str(base_noise_std) + " PSL Model: " + model + " " + metric + " = " + str(np.mean(window_vals)) + " +- " + str(np.std(window_vals)))
            print("Base_oracle_noise_std = " + str(base_noise_std) + " cluster AR " + metric + " = " + str(np.mean(cluster_ar_window_vals)) + " +- " + str(np.std(cluster_ar_window_vals)))
            print("Base_oracle_noise_std = " + str(base_noise_std) + " pure AR " + metric + " = " + str(np.mean(pure_ar_window_vals)) + " +- " + str(np.std(pure_ar_window_vals)) + "\n\n")

Base_oracle_noise_std = 0 PSL Model: cw_hard NRMSE = 0.6865133385523599 +- 0.05013063793315373
Base_oracle_noise_std = 0 cluster AR NRMSE = 0.6238906202656211 +- 0.035599140590143394
Base_oracle_noise_std = 0 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Base_oracle_noise_std = 0.5 PSL Model: cw_hard NRMSE = 0.704174865449105 +- 0.04766166269352869
Base_oracle_noise_std = 0.5 cluster AR NRMSE = 0.641419872424255 +- 0.03669291530776474
Base_oracle_noise_std = 0.5 pure AR NRMSE = 0.7258643355550264 +- 0.037859170384789656


Base_oracle_noise_std = 1 PSL Model: cw_hard NRMSE = 0.7459568385217854 +- 0.050622020530947096
Base_oracle_noise_std = 1 cluster AR NRMSE = 0.6913653097727207 +- 0.03754266559021357
Base_oracle_noise_std = 1 pure AR NRMSE = 0.775530164477302 +- 0.03898970228464027


Base_oracle_noise_std = 1.5 PSL Model: cw_hard NRMSE = 0.8131273727493205 +- 0.062352055278130564
Base_oracle_noise_std = 1.5 cluster AR NRMSE = 0.7656197099534404 +- 0.04029322109849247
Base

In [42]:
# E1: no base noise, vary cluster oracle noise
# Hard constraint, weighted constraints of 10

models = ["temporal_hard"]
temp_or_noise_stds = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

n_folds = 30

results_df = pd.DataFrame(columns=["base_noise_std", "clus_or_std", "temp_or_std", "Series_ID", "Forecast_Window", "Method", "NRMSE", "MASE"])

for temp_or_std in temp_or_noise_stds:
    base_noise_std = 0
    clus_or_std = 0
    
    for model in models:
        print("Evaluating " + model + " with base noise " + str(base_noise_std) + " and cluster oracle noise std. " + str(clus_or_std))
        
        data_dir = "data/E1_nobasenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_" + str(temp_or_std) + "/window_size_4/eval"
        result_dir = "results/Online/E1_nobasenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_" + str(temp_or_std) + "/window_size_4/" + model + "/inferred-predicates"

        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "PSL_" + str(model), results_df, [base_noise_std, clus_or_std, temp_or_std])])
        #results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "cluster_AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True, cluster_baseline=True)])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True)])

Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0
Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0
Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0
Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0
Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0
Evaluating temporal_hard with base noise 0 and cluster oracle noise std. 0


In [44]:
for metric in METRICS:
    for model in models:
        for temp_or_std in temp_or_noise_stds:
            window_vals = []
            cluster_ar_window_vals = []
            pure_ar_window_vals = []

            for name, group in results_df[(results_df["Method"] == "PSL_" + str(model)) & (results_df["base_noise_std"] == 0) & (results_df["clus_or_std"] == 0) & (results_df["temp_or_std"] == temp_or_std)].groupby(by=["Forecast_Window"]):
                window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "AR") & (results_df["base_noise_std"] == 0) & (results_df["clus_or_std"] == 0) & (results_df["temp_or_std"] == temp_or_std)].groupby(by=["Forecast_Window"]):
                pure_ar_window_vals += [np.mean(group[metric].values)]

            print("Temporal_oracle_noise_std = " + str(temp_or_std) + " PSL Model: " + model + " " + metric + " = " + str(np.mean(window_vals)) + " +- " + str(np.std(window_vals)))
            print("Temporal_oracle_noise_std = " + str(temp_or_std) + " pure AR " + metric + " = " + str(np.mean(pure_ar_window_vals)) + " +- " + str(np.std(pure_ar_window_vals)) + "\n\n")

Temporal_oracle_noise_std = 0 PSL Model: temporal_hard NRMSE = 0.7091984463806473 +- 0.037621246387841395
Temporal_oracle_noise_std = 0 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Temporal_oracle_noise_std = 0.1 PSL Model: temporal_hard NRMSE = 0.7089103527264282 +- 0.03766752774040717
Temporal_oracle_noise_std = 0.1 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Temporal_oracle_noise_std = 0.2 PSL Model: temporal_hard NRMSE = 0.7089959119174787 +- 0.03755364676678666
Temporal_oracle_noise_std = 0.2 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Temporal_oracle_noise_std = 0.3 PSL Model: temporal_hard NRMSE = 0.7089250715593675 +- 0.037274318786307305
Temporal_oracle_noise_std = 0.3 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Temporal_oracle_noise_std = 0.4 PSL Model: temporal_hard NRMSE = 0.7089073303567871 +- 0.03721748211561762
Temporal_oracle_noise_std = 0.4 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Temporal_ora

In [47]:
# E1: no base noise, vary cluster oracle noise
# Hard constraint, weighted constraints of 10

models = ["cw_hard_meanprior0.1nsq"]
cluster_or_noise_stds = [0, 0.1, 0.2, 0.3, 0.4]

n_folds = 30

results_df = pd.DataFrame(columns=["base_noise_std", "clus_or_std", "temp_or_std", "Series_ID", "Forecast_Window", "Method", "NRMSE", "MASE"])

for clus_or_std in cluster_or_noise_stds:
    base_noise_std = 0
    temp_or_std = 0
    
    for model in models:
        print("Evaluating " + model + " with base noise " + str(base_noise_std) + " and cluster oracle noise std. " + str(clus_or_std))
        
        data_dir = "data/E1_nobasenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_0/window_size_4/eval"
        result_dir = "results/Online/E1_nobasenoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_or_std) + "/cross_cov_0/temp_or_variance_0/window_size_4/" + model + "/inferred-predicates"

        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "PSL_" + str(model), results_df, [base_noise_std, clus_or_std, temp_or_std])])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "cluster_AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True, cluster_baseline=True)])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "AR", results_df, [base_noise_std, clus_or_std, temp_or_std], eval_baseline=True)])

Evaluating cw_hard_meanprior0.1nsq with base noise 0 and cluster oracle noise std. 0
Evaluating cw_hard_meanprior0.1nsq with base noise 0 and cluster oracle noise std. 0.1
Evaluating cw_hard_meanprior0.1nsq with base noise 0 and cluster oracle noise std. 0.2
Evaluating cw_hard_meanprior0.1nsq with base noise 0 and cluster oracle noise std. 0.3
Evaluating cw_hard_meanprior0.1nsq with base noise 0 and cluster oracle noise std. 0.4


In [10]:
results_df

,base_noise_std,clus_or_std,temp_or_std,Series_ID,Forecast_Window,Method,NRMSE,MASE
0,1,0,0,0,000,PSL_cw_hard,0.903602,0.645074
1,1,0,0,1,000,PSL_cw_hard,0.370476,0.172899
2,1,0,0,2,000,PSL_cw_hard,0.329546,0.219751
3,1,0,0,3,000,PSL_cw_hard,0.358692,0.219198
4,1,0,0,4,000,PSL_cw_hard,0.602820,0.465741
...,...,...,...,...,...,...,...,...
1795,1,1,0,55,029,cluster_AR,0.348639,0.221274
1796,1,1,0,56,029,cluster_AR,0.710280,0.439076
1797,1,1,0,57,029,cluster_AR,1.354515,1.143602
1798,1,1,0,58,029,cluster_AR,0.380426,1.008077


In [50]:
for metric in METRICS:
    for model in models:
        for std in cluster_or_noise_stds:
            window_vals = []
            cluster_ar_window_vals = []
            pure_ar_window_vals = []

            for name, group in results_df[(results_df["Method"] == "PSL_" + str(model)) & (results_df["base_noise_std"] == 0) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "cluster_AR") & (results_df["base_noise_std"] == 0) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                cluster_ar_window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "AR") & (results_df["base_noise_std"] == 0) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                pure_ar_window_vals += [np.mean(group[metric].values)]

            print("Cluster_oracle_noise_std = " + str(std) + " PSL Model: " + model + " " + metric + " = " + str(np.mean(window_vals)) + " +- " + str(np.std(window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " cluster AR " + metric + " = " + str(np.mean(cluster_ar_window_vals)) + " +- " + str(np.std(cluster_ar_window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " pure AR " + metric + " = " + str(np.mean(pure_ar_window_vals)) + " +- " + str(np.std(pure_ar_window_vals)) + "\n\n")

Cluster_oracle_noise_std = 0 PSL Model: cw_hard_meanprior0.1nsq NRMSE = 0.6739318625048952 +- 0.04861992103649635
Cluster_oracle_noise_std = 0 cluster AR NRMSE = 0.6238906202656211 +- 0.035599140590143394
Cluster_oracle_noise_std = 0 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Cluster_oracle_noise_std = 0.1 PSL Model: cw_hard_meanprior0.1nsq NRMSE = 0.6798019140769225 +- 0.04704543991840487
Cluster_oracle_noise_std = 0.1 cluster AR NRMSE = 0.6265974250013367 +- 0.035482410336074445
Cluster_oracle_noise_std = 0.1 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Cluster_oracle_noise_std = 0.2 PSL Model: cw_hard_meanprior0.1nsq NRMSE = 0.6981651994453503 +- 0.04989878962268292
Cluster_oracle_noise_std = 0.2 cluster AR NRMSE = 0.6350245008512952 +- 0.03557058687062343
Cluster_oracle_noise_std = 0.2 pure AR NRMSE = 0.708958242999575 +- 0.03738425007845916


Cluster_oracle_noise_std = 0.3 PSL Model: cw_hard_meanprior0.1nsq NRMSE = 0.7223394765708175 +- 0.051295356713

In [ ]:
for metric in METRICS:
    for model in models:
        for std in cluster_or_noise_stds:
            window_vals = []
            cluster_ar_window_vals = []
            pure_ar_window_vals = []

            for name, group in results_df[(results_df["Method"] == "PSL_" + str(model)) & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "cluster_AR") & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                cluster_ar_window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "AR") & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                pure_ar_window_vals += [np.mean(group[metric].values)]

            print("Cluster_oracle_noise_std = " + str(std) + " PSL Model: " + model + " " + metric + " = " + str(np.mean(window_vals)) + " +- " + str(np.std(window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " cluster AR " + metric + " = " + str(np.mean(cluster_ar_window_vals)) + " +- " + str(np.std(cluster_ar_window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " pure AR " + metric + " = " + str(np.mean(pure_ar_window_vals)) + " +- " + str(np.std(pure_ar_window_vals)) + "\n\n")

In [39]:
# E1: no base noise, vary cluster oracle noise
# Hard constraint, weighted constraints of 10

models = ["cw_hard"]
clus_noise_stds = [0, 1]

n_folds = 30

results_df = pd.DataFrame(columns=["base_noise_std", "clus_or_std", "temp_or_std", "Series_ID", "Forecast_Window", "Method", "NRMSE", "MASE"])

for clus_noise_std in clus_noise_stds:
    base_noise_std = 1
    temp_or_std = 0
    
    for model in models:
        print("Evaluating " + model + " with base noise " + str(base_noise_std) + " and cluster oracle noise std. " + str(clus_noise_std))
        
        data_dir = "data/E1_fixednoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_noise_std) + "/cross_cov_0/temp_or_variance_0/window_size_4/eval"
        result_dir = "results/Online/E1_fixednoise/base_noise_" + str(base_noise_std) + "/clus_or_variance_" + str(clus_noise_std) + "/cross_cov_0/temp_or_variance_0/window_size_4/" + model + "/inferred-predicates"

        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "PSL_" + str(model), results_df, [base_noise_std, clus_noise_std, temp_or_std])])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "cluster_AR", results_df, [base_noise_std, clus_noise_std, temp_or_std], eval_baseline=True, cluster_baseline=True)])
        results_df = pd.concat([results_df, eval_run(data_dir, result_dir, n_folds, "AR", results_df, [base_noise_std, clus_noise_std, temp_or_std], eval_baseline=True)])

Evaluating cw_hard with base noise 1 and cluster oracle noise std. 0
Evaluating cw_hard with base noise 1 and cluster oracle noise std. 1


In [40]:
for metric in METRICS:
    for model in models:
        for std in clus_noise_stds:
            window_vals = []
            cluster_ar_window_vals = []
            pure_ar_window_vals = []

            for name, group in results_df[(results_df["Method"] == "PSL_" + str(model)) & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "cluster_AR") & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                cluster_ar_window_vals += [np.mean(group[metric].values)]
            for name, group in results_df[(results_df["Method"] == "AR") & (results_df["base_noise_std"] == 1) & (results_df["clus_or_std"] == std)].groupby(by=["Forecast_Window"]):
                pure_ar_window_vals += [np.mean(group[metric].values)]

            print("Cluster_oracle_noise_std = " + str(std) + " PSL Model: " + model + " " + metric + " = " + str(np.mean(window_vals)) + " +- " + str(np.std(window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " cluster AR " + metric + " = " + str(np.mean(cluster_ar_window_vals)) + " +- " + str(np.std(cluster_ar_window_vals)))
            print("Cluster_oracle_noise_std = " + str(std) + " pure AR " + metric + " = " + str(np.mean(pure_ar_window_vals)) + " +- " + str(np.std(pure_ar_window_vals)) + "\n\n")

Cluster_oracle_noise_std = 0 PSL Model: cw_hard NRMSE = 0.7441394223649035 +- 0.050528099071055814
Cluster_oracle_noise_std = 0 cluster AR NRMSE = 0.6913653097727207 +- 0.03754266559021357
Cluster_oracle_noise_std = 0 pure AR NRMSE = 0.775530164477302 +- 0.03898970228464027


Cluster_oracle_noise_std = 1 PSL Model: cw_hard NRMSE = 1.1028541010243926 +- 0.0872797098367929
Cluster_oracle_noise_std = 1 cluster AR NRMSE = 0.896661099251259 +- 0.04959188168417733
Cluster_oracle_noise_std = 1 pure AR NRMSE = 0.775530164477302 +- 0.03898970228464027


Cluster_oracle_noise_std = 0 PSL Model: cw_hard SMAPE = 12.076525915635838 +- 1.055149250883876
Cluster_oracle_noise_std = 0 cluster AR SMAPE = 11.250227850281815 +- 0.7339511019130658
Cluster_oracle_noise_std = 0 pure AR SMAPE = 12.859785414203579 +- 0.7945508565800287


Cluster_oracle_noise_std = 1 PSL Model: cw_hard SMAPE = 18.17590128599387 +- 1.3786686551975773
Cluster_oracle_noise_std = 1 cluster AR SMAPE = 14.203375770134093 +- 0.78244908